# Xây Dựng Chương Trình Phân Tích File XML Dùng Cấu Trúc Cây Tổng Quát

## Yêu Cầu Bài Toán

Một file XML bao gồm các thẻ tag:
- Các thẻ tag phải có đóng mở tag, đóng tag
- Các tag có thể có thêm thuộc tính hoặc outerText (text sẽ không chứa các dấu xuống dòng hoặc các dấu cách trống ở đầu và ở cuối)
- Bỏ qua các chú thích và xml version ở đầu file

## Các Tính Năng Cần Có
1. Đọc vào dữ liệu XML từ file và kiểm tra hợp lệ của các thẻ
2. Thêm và xóa thẻ
3. Thay đổi giá trị thuộc tính của thẻ
4. Ghi nội dung hiện tại ra file
5. Tìm kiếm/in ra giá trị của 1 key nào đó trên cây
6. Tìm kiếm và in ra nội dung của 1 thẻ

## Cấu Trúc Dữ Liệu

```c
// Định nghĩa cấu trúc để lưu trữ thuộc tính của thẻ
typedef struct Attribute {
    char* name;
    char* value;
    struct Attribute* next;
} Attribute;

// Định nghĩa cấu trúc nút cây tổng quát cho thẻ XML
typedef struct TreeNode {
    char* tag_name;
    Attribute* attributes;
    char* text;
    struct TreeNode* first_child;
    struct TreeNode* next_sibling;
    struct TreeNode* parent;
} TreeNode;
```

## Lý Thuyết Cây Tổng Quát

### Khái Niệm Cơ Bản
- **Cây (Tree)**: Là kiểu cấu trúc dữ liệu phi tuyến, biểu diễn mối quan hệ có thứ bậc giữa các đối tượng
- **Nút gốc (Root)**: Là nút không có nút nào đứng trên nó
- **Nút lá (Leaf)**: Nút không có nút con
- **Nút trong (Internal node)**: Nút có ít nhất 1 con

### Biểu Diễn Cây: Con Đầu Tiên và Các Nút Anh Chị Em Kế Tiếp

```c
struct TreeNode {
    DATA_TYPE info;
    struct TreeNode *firstChild;    // Con trỏ đến nút con đầu tiên
    struct TreeNode *nextSibling;   // Con trỏ đến nút anh em kế tiếp
    struct TreeNode *parent;        // Bổ sung để thêm và xóa nút nhanh hơn
}
```

### Duyệt Cây
- **Duyệt theo thứ tự trước (Pre-order)**: Xử lý nút hiện tại trước, sau đó xử lý các nút con
- **Duyệt theo thứ tự sau (Post-order)**: Xử lý các nút con trước, sau đó xử lý nút hiện tại
- **Duyệt theo mức (Level-order)**: Dùng hàng đợi để duyệt từng mức

## Chiến Lược Thuật Toán Parser XML

Sử dụng **Ngăn xếp (Stack)** để quản lý "ngữ cảnh" (Context):

### Quy Tắc Xử Lý

1. **Gặp Thẻ Mở** (`<tag attr="value">`)
   - Tạo Node mới với tên thẻ và các thuộc tính
   - Nếu Stack có phần tử: Nút ở đỉnh Stack là Cha, thêm Node mới vào danh sách con
   - Đẩy Node mới vào Stack

2. **Gặp Nội Dung** (text)
   - Gán nội dung vào thuộc tính text của nút đang ở đỉnh Stack

3. **Gặp Thẻ Đóng** (`</tag>`)
   - Kiểm tra tên thẻ đóng có khớp với thẻ mở không
   - Pop nút khỏi Stack

### Ví Dụ Mô Phỏng

Input: `<library><book id="1">Title</book></library>`

| Bước | Đọc được | Stack | Cây |
|------|----------|-------|-----|
| 1 | `<library>` | `[library]` | `library` |
| 2 | `<book id="1">` | `[library, book]` | `library -> book(id=1)` |
| 3 | `Title` | `[library, book]` | `library -> book(text="Title")` |
| 4 | `</book>` | `[library]` | (Đã xong book) |
| 5 | `</library>` | `[]` | (Hoàn thành) |

In [ ]:
import re
from typing import Optional, List, Tuple

# ==========================================
# PHẦN 1: ĐỊNH NGHĨA CẤU TRÚC ATTRIBUTE
# ==========================================

class Attribute:
    """
    Đại diện cho một thuộc tính của thẻ XML.
    Cấu trúc danh sách liên kết đơn.
    
    Thuộc tính:
        name: Tên thuộc tính (VD: id, class)
        value: Giá trị thuộc tính (VD: "1", "main")
        next: Con trỏ đến thuộc tính tiếp theo
    """
    def __init__(self, name: str, value: str):
        self.name = name      # Tên thuộc tính
        self.value = value    # Giá trị thuộc tính
        self.next = None      # Con trỏ đến thuộc tính tiếp theo
    
    def __repr__(self):
        return f'{self.name}="{self.value}"'

In [ ]:
# ==========================================
# PHẦN 2: ĐỊNH NGHĨA CẤU TRÚC TREENODE
# ==========================================

class TreeNode:
    """
    Đại diện cho một nút trong cây tổng quát XML.
    Sử dụng mô hình: First Child - Next Sibling - Parent
    
    Thuộc tính:
        tag_name: Tên thẻ (VD: library, book)
        attributes: Danh sách liên kết các thuộc tính
        text: Nội dung văn bản bên trong thẻ
        first_child: Con trỏ đến nút con đầu tiên
        next_sibling: Con trỏ đến nút anh em kế tiếp
        parent: Con trỏ đến nút cha
    """
    def __init__(self, tag_name: str):
        self.tag_name = tag_name           # Tên thẻ
        self.attributes = None             # Danh sách liên kết các thuộc tính
        self.text = ""                     # Nội dung văn bản
        self.first_child = None            # Con trỏ đến nút con đầu tiên
        self.next_sibling = None           # Con trỏ đến nút anh em kế tiếp
        self.parent = None                 # Con trỏ đến nút cha
    
    def __repr__(self):
        return f"<TreeNode: {self.tag_name}>"

In [ ]:
# ==========================================
# PHẦN 3: CÁC HÀM THAO TÁC VỚI ATTRIBUTE
# ==========================================

def add_attribute(node: TreeNode, name: str, value: str):
    """
    Thêm một thuộc tính mới vào danh sách thuộc tính của node.
    Thuộc tính mới sẽ được thêm vào cuối danh sách liên kết.
    
    Tham số:
        node: Nút cần thêm thuộc tính
        name: Tên thuộc tính
        value: Giá trị thuộc tính
    """
    # Tạo thuộc tính mới
    new_attr = Attribute(name, value)
    
    if node.attributes is None:
        # Nếu chưa có thuộc tính nào, đây là thuộc tính đầu tiên
        node.attributes = new_attr
    else:
        # Di chuyển đến cuối danh sách liên kết
        current = node.attributes
        while current.next is not None:
            current = current.next
        # Thêm thuộc tính mới vào cuối
        current.next = new_attr


def get_attribute(node: TreeNode, name: str) -> Optional[str]:
    """
    Lấy giá trị của thuộc tính theo tên.
    
    Tham số:
        node: Nút cần tìm thuộc tính
        name: Tên thuộc tính cần tìm
    
    Trả về:
        Giá trị thuộc tính nếu tìm thấy, None nếu không tìm thấy
    """
    current = node.attributes
    while current is not None:
        if current.name == name:
            return current.value
        current = current.next
    return None


def set_attribute(node: TreeNode, name: str, new_value: str) -> bool:
    """
    Thay đổi giá trị của thuộc tính theo tên.
    
    Tham số:
        node: Nút cần sửa thuộc tính
        name: Tên thuộc tính cần sửa
        new_value: Giá trị mới
    
    Trả về:
        True nếu thành công, False nếu không tìm thấy thuộc tính
    """
    current = node.attributes
    while current is not None:
        if current.name == name:
            current.value = new_value
            return True
        current = current.next
    return False


def remove_attribute(node: TreeNode, name: str) -> bool:
    """
    Xóa thuộc tính theo tên.
    
    Tham số:
        node: Nút cần xóa thuộc tính
        name: Tên thuộc tính cần xóa
    
    Trả về:
        True nếu thành công, False nếu không tìm thấy
    """
    if node.attributes is None:
        return False
    
    # Trường hợp xóa phần tử đầu tiên
    if node.attributes.name == name:
        node.attributes = node.attributes.next
        return True
    
    # Tìm và xóa trong danh sách
    prev = node.attributes
    current = prev.next
    while current is not None:
        if current.name == name:
            prev.next = current.next
            return True
        prev = current
        current = current.next
    return False


def get_all_attributes(node: TreeNode) -> List[Tuple[str, str]]:
    """
    Lấy danh sách tất cả các thuộc tính của node.
    
    Tham số:
        node: Nút cần lấy thuộc tính
    
    Trả về:
        Danh sách các tuple (tên, giá trị)
    """
    result = []
    current = node.attributes
    while current is not None:
        result.append((current.name, current.value))
        current = current.next
    return result

In [ ]:
# ==========================================
# PHẦN 4: CÁC HÀM THAO TÁC VỚI CÂY (TREENODE)
# ==========================================

def add_child(parent: TreeNode, child: TreeNode):
    """
    Thêm một nút con vào cuối danh sách con của parent.
    Sử dụng mô hình first_child - next_sibling.
    
    Tham số:
        parent: Nút cha
        child: Nút con cần thêm
    """
    # Thiết lập quan hệ cha-con
    child.parent = parent
    
    if parent.first_child is None:
        # Chưa có con nào, child là con đầu tiên
        parent.first_child = child
    else:
        # Tìm con cuối cùng (rightmost sibling)
        current = parent.first_child
        while current.next_sibling is not None:
            current = current.next_sibling
        # Thêm child vào sau con cuối cùng
        current.next_sibling = child


def remove_child(parent: TreeNode, child: TreeNode) -> bool:
    """
    Xóa một nút con khỏi parent.
    
    Tham số:
        parent: Nút cha
        child: Nút con cần xóa
    
    Trả về:
        True nếu thành công, False nếu không tìm thấy
    """
    if parent.first_child is None:
        return False
    
    # Trường hợp xóa con đầu tiên
    if parent.first_child == child:
        parent.first_child = child.next_sibling
        child.parent = None
        child.next_sibling = None
        return True
    
    # Tìm và xóa trong danh sách siblings
    prev = parent.first_child
    current = prev.next_sibling
    while current is not None:
        if current == child:
            prev.next_sibling = current.next_sibling
            child.parent = None
            child.next_sibling = None
            return True
        prev = current
        current = current.next_sibling
    return False


def find_node_by_tag(root: TreeNode, tag_name: str) -> Optional[TreeNode]:
    """
    Tìm nút đầu tiên có tên thẻ trùng khớp (duyệt pre-order).
    
    Tham số:
        root: Nút gốc của cây
        tag_name: Tên thẻ cần tìm
    
    Trả về:
        Nút tìm được hoặc None nếu không tìm thấy
    """
    if root is None:
        return None
    
    # Kiểm tra nút hiện tại
    if root.tag_name == tag_name:
        return root
    
    # Duyệt qua các con (đệ quy)
    child = root.first_child
    while child is not None:
        result = find_node_by_tag(child, tag_name)
        if result is not None:
            return result
        child = child.next_sibling
    
    return None


def find_all_nodes_by_tag(root: TreeNode, tag_name: str) -> List[TreeNode]:
    """
    Tìm tất cả các nút có tên thẻ trùng khớp.
    
    Tham số:
        root: Nút gốc của cây
        tag_name: Tên thẻ cần tìm
    
    Trả về:
        Danh sách các nút tìm được
    """
    result = []
    
    def traverse(node):
        """Hàm duyệt cây đệ quy theo thứ tự trước (pre-order)"""
        if node is None:
            return
        # Kiểm tra nút hiện tại
        if node.tag_name == tag_name:
            result.append(node)
        # Duyệt các con
        child = node.first_child
        while child is not None:
            traverse(child)
            child = child.next_sibling
    
    traverse(root)
    return result


def count_nodes(root: TreeNode) -> int:
    """
    Đếm số lượng nút trên cây.
    
    Tham số:
        root: Nút gốc của cây
    
    Trả về:
        Số lượng nút
    """
    if root is None:
        return 0
    
    count = 1  # Đếm nút hiện tại
    # Đệ quy đếm các nút con
    child = root.first_child
    while child is not None:
        count += count_nodes(child)
        child = child.next_sibling
    
    return count


def get_tree_height(root: TreeNode) -> int:
    """
    Tính chiều cao của cây (chiều cao nút lá = 0).
    
    Tham số:
        root: Nút gốc của cây
    
    Trả về:
        Chiều cao của cây
    """
    if root is None:
        return -1  # Cây rỗng
    
    if root.first_child is None:
        return 0  # Nút lá có chiều cao 0
    
    # Tìm chiều cao lớn nhất trong các cây con
    max_height = 0
    child = root.first_child
    while child is not None:
        h = get_tree_height(child)
        if h > max_height:
            max_height = h
        child = child.next_sibling
    
    return max_height + 1

In [ ]:
# ==========================================
# PHẦN 5: XML PARSER - ĐỌC VÀ KIỂM TRA HỢP LỆ
# ==========================================

class XMLParseError(Exception):
    """Lớp Exception cho lỗi phân tích XML."""
    pass


def parse_attributes(attr_string: str) -> List[Tuple[str, str]]:
    """
    Phân tích chuỗi thuộc tính thành danh sách (tên, giá trị).
    
    Ví dụ:
        'id="1" class="main"' -> [('id', '1'), ('class', 'main')]
    
    Tham số:
        attr_string: Chuỗi chứa các thuộc tính
    
    Trả về:
        Danh sách các tuple (tên, giá trị)
    """
    attrs = []
    # Regex để bắt cặp name="value" hoặc name='value'
    pattern = r'([\w-]+)\s*=\s*["\']([^"\']*)["\']'
    matches = re.findall(pattern, attr_string)
    for name, value in matches:
        attrs.append((name, value))
    return attrs


def tokenize_xml(xml_string: str) -> List[Tuple[str, str]]:
    """
    Tách chuỗi XML thành các token.
    
    Tham số:
        xml_string: Chuỗi XML cần tách
    
    Trả về:
        Danh sách các tuple (loại_token, giá_trị):
        - ('open', '<tag attr="val">') - Thẻ mở
        - ('close', '</tag>') - Thẻ đóng
        - ('text', 'nội dung') - Nội dung văn bản
        - ('self_close', '<tag />') - Thẻ tự đóng
    """
    tokens = []
    
    # Bỏ qua XML declaration và comments
    xml_string = re.sub(r'<\?xml[^?]*\?>', '', xml_string)
    xml_string = re.sub(r'<!--[\s\S]*?-->', '', xml_string)
    
    # Regex để tách các thành phần
    pattern = r'(<[^>]+>)|([^<]+)'
    
    for match in re.finditer(pattern, xml_string):
        tag = match.group(1)   # Thẻ XML
        text = match.group(2)  # Nội dung văn bản
        
        if tag:
            tag = tag.strip()
            if tag.startswith('</'):
                # Thẻ đóng
                tokens.append(('close', tag))
            elif tag.endswith('/>'):
                # Thẻ tự đóng
                tokens.append(('self_close', tag))
            else:
                # Thẻ mở
                tokens.append(('open', tag))
        elif text:
            # Loại bỏ khoảng trắng ở đầu và cuối
            text = text.strip()
            if text:  # Chỉ thêm nếu không rỗng
                tokens.append(('text', text))
    
    return tokens


def extract_tag_info(tag_string: str) -> Tuple[str, List[Tuple[str, str]]]:
    """
    Trích xuất tên thẻ và các thuộc tính từ chuỗi thẻ.
    
    Ví dụ:
        '<book id="1">' -> ('book', [('id', '1')])
    
    Tham số:
        tag_string: Chuỗi thẻ cần phân tích
    
    Trả về:
        Tuple (tên_thẻ, danh_sách_thuộc_tính)
    """
    # Loại bỏ < và >
    content = tag_string.strip('<>/')
    
    # Tách tên thẻ và phần thuộc tính
    parts = content.split(None, 1)  # Tách theo khoảng trắng đầu tiên
    tag_name = parts[0]
    
    attrs = []
    if len(parts) > 1:
        attrs = parse_attributes(parts[1])
    
    return tag_name, attrs


def parse_xml_to_tree(xml_string: str) -> TreeNode:
    """
    Chuyển đổi chuỗi XML thành Cây tổng quát.
    Kiểm tra tính hợp lệ của các thẻ (mở/đóng phải khớp).
    
    Tham số:
        xml_string: Chuỗi XML cần phân tích
    
    Trả về:
        Nút gốc (Root) của cây
    
    Ngoại lệ:
        XMLParseError: Nếu XML không hợp lệ
    """
    # Bước 1: Tách chuỗi thành các token
    tokens = tokenize_xml(xml_string)
    
    if not tokens:
        raise XMLParseError("XML rỗng hoặc không hợp lệ")
    
    root = None
    stack = []  # Stack lưu các TreeNode đang xử lý
    
    # Bước 2: Duyệt qua từng token và xây dựng cây
    for token_type, token_value in tokens:
        
        if token_type == 'open':
            # Gặp thẻ mở: Tạo nút mới và thêm vào cây
            tag_name, attrs = extract_tag_info(token_value)
            
            # Tạo nút mới
            new_node = TreeNode(tag_name)
            
            # Thêm các thuộc tính
            for attr_name, attr_value in attrs:
                add_attribute(new_node, attr_name, attr_value)
            
            # Xác định root hoặc thêm làm con
            if root is None:
                root = new_node
            
            if stack:
                # Có nút cha trong stack, thêm làm con
                current_parent = stack[-1]
                add_child(current_parent, new_node)
            
            # Đẩy nút mới vào stack
            stack.append(new_node)
        
        elif token_type == 'close':
            # Gặp thẻ đóng: Kiểm tra và pop khỏi stack
            close_tag_name = token_value.strip('</>')
            
            if not stack:
                raise XMLParseError(
                    f"Gặp thẻ đóng '{close_tag_name}' nhưng không có thẻ mở tương ứng"
                )
            
            # Kiểm tra khớp tên thẻ
            current_node = stack[-1]
            if current_node.tag_name != close_tag_name:
                raise XMLParseError(
                    f"Thẻ đóng '{close_tag_name}' không khớp với thẻ mở '{current_node.tag_name}'"
                )
            
            # Pop khỏi stack (đã xử lý xong nút này)
            stack.pop()
        
        elif token_type == 'self_close':
            # Thẻ tự đóng: <tag />
            tag_name, attrs = extract_tag_info(token_value)
            new_node = TreeNode(tag_name)
            
            for attr_name, attr_value in attrs:
                add_attribute(new_node, attr_name, attr_value)
            
            if root is None:
                root = new_node
            
            if stack:
                current_parent = stack[-1]
                add_child(current_parent, new_node)
        
        elif token_type == 'text':
            # Gán nội dung văn bản cho nút hiện tại
            if stack:
                current_node = stack[-1]
                current_node.text = token_value
    
    # Kiểm tra còn thẻ chưa đóng
    if stack:
        unclosed_tags = [node.tag_name for node in stack]
        raise XMLParseError(f"Các thẻ chưa được đóng: {unclosed_tags}")
    
    return root


def parse_xml_from_file(filepath: str) -> TreeNode:
    """
    Đọc và phân tích XML từ file.
    
    Tham số:
        filepath: Đường dẫn đến file XML
    
    Trả về:
        Nút gốc của cây
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        xml_content = f.read()
    return parse_xml_to_tree(xml_content)

In [ ]:
# ==========================================
# PHẦN 6: HIỂN THỊ CÂY XML
# ==========================================

def display_tree(node: TreeNode, level: int = 0, prefix: str = ""):
    """
    Hiển thị cây XML theo dạng cấu trúc dễ đọc.
    
    Tham số:
        node: Nút cần hiển thị
        level: Cấp độ hiện tại (dùng để thụt lề)
        prefix: Tiền tố hiển thị
    """
    if node is None:
        return
    
    # Tạo khoảng trắng thụt lề
    indent = "  " * level
    
    # Xây dựng chuỗi thuộc tính
    attrs_str = ""
    attrs = get_all_attributes(node)
    if attrs:
        attrs_str = " " + " ".join([f'{name}="{value}"' for name, value in attrs])
    
    # Hiển thị nội dung nếu có
    text_str = ""
    if node.text:
        text_str = f": {node.text}"
    
    # In ra dòng hiện tại
    print(f"{indent}{prefix}<{node.tag_name}{attrs_str}>{text_str}")
    
    # Đệ quy hiển thị các con
    child = node.first_child
    while child is not None:
        display_tree(child, level + 1, "|- ")
        child = child.next_sibling


def print_tree_structure(root: TreeNode):
    """
    In cấu trúc cây với thông tin chi tiết.
    
    Tham số:
        root: Nút gốc của cây
    """
    print("=" * 50)
    print("CẤU TRÚC CÂY XML")
    print("=" * 50)
    display_tree(root)
    print("=" * 50)
    print(f"Tổng số nút: {count_nodes(root)}")
    print(f"Chiều cao cây: {get_tree_height(root)}")
    print("=" * 50)

In [ ]:
# ==========================================
# PHẦN 7: GHI CÂY XML RA FILE
# ==========================================

def tree_to_xml_string(node: TreeNode, level: int = 0) -> str:
    """
    Chuyển đổi cây thành chuỗi XML.
    
    Tham số:
        node: Nút cần chuyển đổi
        level: Cấp độ hiện tại (dùng để thụt lề)
    
    Trả về:
        Chuỗi XML
    """
    if node is None:
        return ""
    
    indent = "  " * level
    result = ""
    
    # Xây dựng phần thuộc tính
    attrs_str = ""
    attrs = get_all_attributes(node)
    if attrs:
        attrs_str = " " + " ".join([f'{name}="{value}"' for name, value in attrs])
    
    # Kiểm tra có con hoặc text không
    has_children = node.first_child is not None
    has_text = bool(node.text)
    
    if not has_children and not has_text:
        # Thẻ rỗng
        result = f"{indent}<{node.tag_name}{attrs_str}></{node.tag_name}>\n"
    elif has_text and not has_children:
        # Chỉ có text, viết trên 1 dòng
        result = f"{indent}<{node.tag_name}{attrs_str}>{node.text}</{node.tag_name}>\n"
    else:
        # Có con hoặc cả hai
        result = f"{indent}<{node.tag_name}{attrs_str}>\n"
        
        if has_text:
            result += f"{indent}  {node.text}\n"
        
        # Đệ quy xử lý các con
        child = node.first_child
        while child is not None:
            result += tree_to_xml_string(child, level + 1)
            child = child.next_sibling
        
        result += f"{indent}</{node.tag_name}>\n"
    
    return result


def write_xml_to_file(root: TreeNode, filepath: str, include_declaration: bool = True):
    """
    Ghi cây XML ra file.
    
    Tham số:
        root: Nút gốc của cây
        filepath: Đường dẫn file đích
        include_declaration: Có thêm khai báo XML ở đầu file không
    """
    with open(filepath, 'w', encoding='utf-8') as f:
        if include_declaration:
            f.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        f.write(tree_to_xml_string(root))
    print(f"Đã ghi XML ra file: {filepath}")

In [ ]:
# ==========================================
# PHẦN 8: TÌM KIẾM TRÊN CÂY XML
# ==========================================

def find_value_by_key(root: TreeNode, key: str) -> List[str]:
    """
    Tìm kiếm và trả về tất cả giá trị của các thẻ có tên là 'key'.
    
    Tham số:
        root: Nút gốc của cây
        key: Tên thẻ cần tìm
    
    Trả về:
        Danh sách text content của các thẻ tìm được
    """
    results = []
    nodes = find_all_nodes_by_tag(root, key)
    for node in nodes:
        if node.text:
            results.append(node.text)
    return results


def print_values_by_key(root: TreeNode, key: str):
    """
    In ra tất cả giá trị của các thẻ có tên là 'key'.
    
    Tham số:
        root: Nút gốc của cây
        key: Tên thẻ cần tìm
    """
    values = find_value_by_key(root, key)
    print(f"\nTìm kiếm thẻ <{key}>:")
    print("-" * 40)
    if values:
        for i, val in enumerate(values, 1):
            print(f"  {i}. {val}")
    else:
        print(f"  Không tìm thấy thẻ <{key}> nào có nội dung.")
    print("-" * 40)


def find_node_by_attribute(root: TreeNode, attr_name: str, attr_value: str) -> Optional[TreeNode]:
    """
    Tìm nút đầu tiên có thuộc tính với giá trị cho trước.
    
    Tham số:
        root: Nút gốc của cây
        attr_name: Tên thuộc tính
        attr_value: Giá trị thuộc tính cần tìm
    
    Trả về:
        Nút tìm được hoặc None
    """
    if root is None:
        return None
    
    # Kiểm tra nút hiện tại
    if get_attribute(root, attr_name) == attr_value:
        return root
    
    # Đệ quy tìm trong các con
    child = root.first_child
    while child is not None:
        result = find_node_by_attribute(child, attr_name, attr_value)
        if result is not None:
            return result
        child = child.next_sibling
    
    return None


def print_tag_content(root: TreeNode, tag_name: str):
    """
    Tìm và in ra nội dung đầy đủ của một thẻ (bao gồm các thẻ con).
    
    Tham số:
        root: Nút gốc của cây
        tag_name: Tên thẻ cần in nội dung
    """
    node = find_node_by_tag(root, tag_name)
    print(f"\nNội dung thẻ <{tag_name}>:")
    print("-" * 40)
    if node:
        display_tree(node)
    else:
        print(f"  Không tìm thấy thẻ <{tag_name}>.")
    print("-" * 40)

In [ ]:
# ==========================================
# PHẦN 9: THÊM VÀ XÓA THẺ
# ==========================================

def insert_tag(root: TreeNode, parent_tag: str, new_tag_name: str, 
               text: str = "", attributes: dict = None) -> bool:
    """
    Thêm một thẻ mới vào trong một thẻ cha.
    
    Tham số:
        root: Gốc cây
        parent_tag: Tên thẻ cha cần thêm thẻ mới vào
        new_tag_name: Tên thẻ mới
        text: Nội dung văn bản của thẻ mới
        attributes: Dictionary các thuộc tính {tên: giá_trị}
    
    Trả về:
        True nếu thành công, False nếu không tìm thấy thẻ cha
    """
    # Tìm thẻ cha
    parent_node = find_node_by_tag(root, parent_tag)
    if parent_node is None:
        print(f"Không tìm thấy thẻ cha <{parent_tag}>")
        return False
    
    # Tạo nút mới
    new_node = TreeNode(new_tag_name)
    new_node.text = text
    
    # Thêm thuộc tính
    if attributes:
        for name, value in attributes.items():
            add_attribute(new_node, name, value)
    
    # Thêm vào cây
    add_child(parent_node, new_node)
    print(f"Đã thêm thẻ <{new_tag_name}> vào <{parent_tag}>")
    return True


def delete_tag(root: TreeNode, tag_name: str) -> Tuple[bool, Optional[TreeNode]]:
    """
    Xóa thẻ đầu tiên tìm thấy khỏi cây.
    Không thể xóa nút gốc.
    
    Tham số:
        root: Gốc cây
        tag_name: Tên thẻ cần xóa
    
    Trả về:
        Tuple (thành_công, gốc_mới)
    """
    if root is None:
        return False, root
    
    # Không cho phép xóa nút gốc
    if root.tag_name == tag_name:
        print(f"Không thể xóa nút gốc <{tag_name}>")
        return False, root
    
    # Tìm nút cần xóa
    node_to_delete = find_node_by_tag(root, tag_name)
    if node_to_delete is None:
        print(f"Không tìm thấy thẻ <{tag_name}> để xóa")
        return False, root
    
    # Xóa khỏi nút cha
    parent = node_to_delete.parent
    if parent:
        success = remove_child(parent, node_to_delete)
        if success:
            print(f"Đã xóa thẻ <{tag_name}>")
        return success, root
    
    return False, root


def delete_tag_by_attribute(root: TreeNode, attr_name: str, attr_value: str) -> Tuple[bool, Optional[TreeNode]]:
    """
    Xóa thẻ có thuộc tính với giá trị cho trước.
    
    Tham số:
        root: Gốc cây
        attr_name: Tên thuộc tính
        attr_value: Giá trị thuộc tính
    
    Trả về:
        Tuple (thành_công, gốc_mới)
    """
    # Tìm nút có thuộc tính tương ứng
    node_to_delete = find_node_by_attribute(root, attr_name, attr_value)
    if node_to_delete is None:
        print(f"Không tìm thấy thẻ có {attr_name}=\"{attr_value}\"")
        return False, root
    
    # Không cho phép xóa nút gốc
    if node_to_delete == root:
        print("Không thể xóa nút gốc")
        return False, root
    
    # Xóa khỏi nút cha
    parent = node_to_delete.parent
    if parent:
        success = remove_child(parent, node_to_delete)
        if success:
            print(f"Đã xóa thẻ <{node_to_delete.tag_name}> có {attr_name}=\"{attr_value}\"")
        return success, root
    
    return False, root

In [ ]:
# ==========================================
# PHẦN 10: THAY ĐỔI GIÁ TRỊ THUỘC TÍNH
# ==========================================

def modify_attribute(root: TreeNode, tag_name: str, attr_name: str, new_value: str) -> bool:
    """
    Thay đổi giá trị thuộc tính của một thẻ.
    
    Tham số:
        root: Gốc cây
        tag_name: Tên thẻ cần sửa
        attr_name: Tên thuộc tính cần sửa
        new_value: Giá trị mới
    
    Trả về:
        True nếu thành công
    """
    # Tìm nút
    node = find_node_by_tag(root, tag_name)
    if node is None:
        print(f"Không tìm thấy thẻ <{tag_name}>")
        return False
    
    # Cập nhật thuộc tính
    success = set_attribute(node, attr_name, new_value)
    if success:
        print(f"Đã cập nhật {attr_name}=\"{new_value}\" cho thẻ <{tag_name}>")
    else:
        print(f"Không tìm thấy thuộc tính '{attr_name}' trong thẻ <{tag_name}>")
    return success


def add_new_attribute(root: TreeNode, tag_name: str, attr_name: str, attr_value: str) -> bool:
    """
    Thêm thuộc tính mới cho một thẻ.
    
    Tham số:
        root: Gốc cây
        tag_name: Tên thẻ cần thêm thuộc tính
        attr_name: Tên thuộc tính mới
        attr_value: Giá trị thuộc tính
    
    Trả về:
        True nếu thành công
    """
    # Tìm nút
    node = find_node_by_tag(root, tag_name)
    if node is None:
        print(f"Không tìm thấy thẻ <{tag_name}>")
        return False
    
    # Thêm thuộc tính
    add_attribute(node, attr_name, attr_value)
    print(f"Đã thêm thuộc tính {attr_name}=\"{attr_value}\" cho thẻ <{tag_name}>")
    return True


def modify_text(root: TreeNode, tag_name: str, new_text: str) -> bool:
    """
    Thay đổi nội dung văn bản của một thẻ.
    
    Tham số:
        root: Gốc cây
        tag_name: Tên thẻ cần sửa
        new_text: Nội dung mới
    
    Trả về:
        True nếu thành công
    """
    # Tìm nút
    node = find_node_by_tag(root, tag_name)
    if node is None:
        print(f"Không tìm thấy thẻ <{tag_name}>")
        return False
    
    # Cập nhật nội dung
    old_text = node.text
    node.text = new_text
    print(f"Đã cập nhật nội dung thẻ <{tag_name}>: '{old_text}' -> '{new_text}'")
    return True

## Demo Chương Trình

Phần này minh họa cách sử dụng các hàm đã định nghĩa.

In [ ]:
# ==========================================
# DEMO: DỮ LIỆU XML MẪU
# ==========================================

sample_xml = """
<?xml version="1.0" encoding="UTF-8"?>
<library>
    <book id="1">
        <title>Introduction to Algorithms</title>
        <author>
            <first_name>Thomas</first_name>
            <last_name>Cormen</last_name>
        </author>
        <publisher>MIT Press</publisher>
        <year>2009</year>
        <price>89.99</price>
    </book>
    <book id="2">
        <title>Data Structures and Algorithms</title>
        <author>
            <first_name>Robert</first_name>
            <last_name>Sedgewick</last_name>
        </author>
        <publisher>Addison-Wesley</publisher>
        <year>2011</year>
        <price>75.00</price>
    </book>
</library>
"""

print("=" * 60)
print("CHƯƠNG TRÌNH PHÂN TÍCH XML DÙNG CÂY TỔNG QUÁT")
print("=" * 60)

In [ ]:
# ==========================================
# TÍNH NĂNG 1: ĐỌC VÀ PHÂN TÍCH XML
# ==========================================

print("\n[1] ĐỌC VÀ PHÂN TÍCH XML")
print("-" * 60)

try:
    # Phân tích XML và xây dựng cây
    root = parse_xml_to_tree(sample_xml)
    print("Phân tích XML thành công!")
    print(f"Nút gốc: <{root.tag_name}>")
except XMLParseError as e:
    print(f"Lỗi phân tích XML: {e}")

In [ ]:
# ==========================================
# HIỂN THỊ CẤU TRÚC CÂY
# ==========================================

print("\n[HIỂN THỊ CẤU TRÚC CÂY]")
print_tree_structure(root)

In [ ]:
# ==========================================
# TÍNH NĂNG 5: TÌM KIẾM GIÁ TRỊ THEO KEY
# ==========================================

print("\n[5] TÌM KIẾM GIÁ TRỊ CỦA KEY TRÊN CÂY")
print("-" * 60)

# Tìm tất cả title
print_values_by_key(root, "title")

# Tìm tất cả price
print_values_by_key(root, "price")

# Tìm tất cả first_name
print_values_by_key(root, "first_name")

In [ ]:
# ==========================================
# TÍNH NĂNG 6: TÌM VÀ IN NỘI DUNG THẺ
# ==========================================

print("\n[6] TÌM VÀ IN NỘI DUNG CỦA THẺ")
print("-" * 60)

# In nội dung thẻ author đầu tiên
print_tag_content(root, "author")

# In nội dung thẻ book đầu tiên
print_tag_content(root, "book")

In [ ]:
# ==========================================
# TÍNH NĂNG 2: THÊM THẺ MỚI
# ==========================================

print("\n[2] THÊM THẺ MỚI")
print("-" * 60)

# Thêm sách mới vào library
insert_tag(
    root, 
    parent_tag="library", 
    new_tag_name="book", 
    attributes={"id": "3"}
)

# Tìm sách vừa thêm và thêm các thẻ con
new_book = find_node_by_attribute(root, "id", "3")
if new_book:
    # Thêm title
    title_node = TreeNode("title")
    title_node.text = "The Art of Computer Programming"
    add_child(new_book, title_node)
    
    # Thêm author
    author_node = TreeNode("author")
    add_child(new_book, author_node)
    
    # Thêm first_name và last_name vào author
    first_name = TreeNode("first_name")
    first_name.text = "Donald"
    add_child(author_node, first_name)
    
    last_name = TreeNode("last_name")
    last_name.text = "Knuth"
    add_child(author_node, last_name)
    
    # Thêm price
    price_node = TreeNode("price")
    price_node.text = "150.00"
    add_child(new_book, price_node)
    
    print("Đã thêm đầy đủ thông tin sách mới!")

print("\nCấu trúc cây sau khi thêm:")
print_tree_structure(root)

In [ ]:
# ==========================================
# TÍNH NĂNG 3: THAY ĐỔI THUỘC TÍNH
# ==========================================

print("\n[3] THAY ĐỔI GIÁ TRỊ THUỘC TÍNH")
print("-" * 60)

# Tìm book có id="1" và đổi thành id="100"
book1 = find_node_by_attribute(root, "id", "1")
if book1:
    print(f"Tìm thấy thẻ <book> với id=\"1\"")
    set_attribute(book1, "id", "100")
    print("Đã đổi id từ \"1\" thành \"100\"")

# Thêm thuộc tính mới cho thẻ library
add_new_attribute(root, "library", "name", "HUST Library")

# Thay đổi nội dung text của thẻ price đầu tiên
modify_text(root, "price", "99.99")

print("\nCấu trúc cây sau khi thay đổi:")
display_tree(root)

In [ ]:
# ==========================================
# TÍNH NĂNG 2 (TIẾP): XÓA THẺ
# ==========================================

print("\n[2] XÓA THẺ")
print("-" * 60)

# Xóa sách có id="2"
success, root = delete_tag_by_attribute(root, "id", "2")

print("\nCấu trúc cây sau khi xóa:")
print_tree_structure(root)

In [ ]:
# ==========================================
# TÍNH NĂNG 4: GHI RA FILE
# ==========================================

print("\n[4] GHI NỘI DUNG RA FILE")
print("-" * 60)

# Hiển thị XML string
xml_output = tree_to_xml_string(root)
print("Nội dung XML:")
print(xml_output)

# Ghi ra file
write_xml_to_file(root, "output.xml")

In [ ]:
# ==========================================
# KIỂM TRA TÍNH HỢP LỆ CỦA XML
# ==========================================

print("\n[KIỂM TRA TÍNH HỢP LỆ]")
print("-" * 60)

# Test 1: XML không hợp lệ - thiếu thẻ đóng
invalid_xml1 = "<root><child>text</root>"
print("\nTest 1: XML thiếu thẻ đóng")
print(f"Input: {invalid_xml1}")
try:
    parse_xml_to_tree(invalid_xml1)
except XMLParseError as e:
    print(f"Lỗi: {e}")

# Test 2: XML không hợp lệ - thẻ đóng không khớp
invalid_xml2 = "<root><child></wrong></root>"
print("\nTest 2: XML thẻ đóng không khớp")
print(f"Input: {invalid_xml2}")
try:
    parse_xml_to_tree(invalid_xml2)
except XMLParseError as e:
    print(f"Lỗi: {e}")

# Test 3: XML hợp lệ
valid_xml = "<root><child>text</child></root>"
print("\nTest 3: XML hợp lệ")
print(f"Input: {valid_xml}")
try:
    test_root = parse_xml_to_tree(valid_xml)
    print("Phân tích thành công!")
    display_tree(test_root)
except XMLParseError as e:
    print(f"Lỗi: {e}")

## Tổng Kết

Chương trình đã cài đặt đầy đủ các tính năng yêu cầu:

| STT | Tính Năng | Hàm Chính |
|-----|-----------|------------|
| 1 | Đọc XML từ file và kiểm tra hợp lệ | `parse_xml_to_tree()`, `parse_xml_from_file()` |
| 2 | Thêm thẻ | `insert_tag()`, `add_child()` |
| 2 | Xóa thẻ | `delete_tag()`, `delete_tag_by_attribute()` |
| 3 | Thay đổi thuộc tính | `modify_attribute()`, `set_attribute()` |
| 4 | Ghi ra file | `write_xml_to_file()`, `tree_to_xml_string()` |
| 5 | Tìm kiếm giá trị key | `find_value_by_key()`, `print_values_by_key()` |
| 6 | In nội dung thẻ | `print_tag_content()`, `display_tree()` |

### Cấu Trúc Dữ Liệu
- **Attribute**: Danh sách liên kết đơn chứa các thuộc tính (name, value, next)
- **TreeNode**: Cây tổng quát với cấu trúc first_child - next_sibling - parent

### Thuật Toán Chính
- Sử dụng **Stack** để parse XML, đảm bảo kiểm tra tính hợp lệ của các thẻ mở/đóng
- Sử dụng **Đệ quy** để duyệt cây (pre-order traversal)
- Sử dụng **Regex** để tách các token và thuộc tính từ chuỗi XML